In [2]:
#Do all of the imports and setup inline plotting
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

from TDA import *

def getSlidingWindow(x, dim, Tau):
    N = len(x)
    NWindows = N-dim+1
    X = np.zeros((NWindows, dim))
    maxIndex = 0
    for i in range(NWindows):
        if len(x[i:i+Tau*dim:Tau]) < dim:
            #If the window goes out of bounds, simply break
            #and discard those windows
            break
        X[i, :] = x[i:i+Tau*dim:Tau]
        maxIndex += 1
    return X[0:maxIndex, :]

In [3]:
#Step 1: Setup the signal
T1 = 30 #The period of the first sine in number of samples
T2 = T1/np.pi
NPeriods = 10 #How many periods to go through, relative to the first sinusoid
N = T1*NPeriods #The total number of samples
t = np.arange(N) #Time indices
x = np.cos(2*np.pi*(1.0/T1)*t) #The first sinusoid
x += np.cos(2*np.pi*(1.0/T2)*t) #The second sinusoid
#TODO: Separate signal plot
plt.figure()
plt.plot(x)
plt.show()

In [6]:
#Step 2: Do a sliding window embedding
dim = 5
Tau = 6
X = getSlidingWindow(x, dim, Tau)

#Step 2.5: Do Rips Filtration
PDs = doRipsFiltration(X, 1)

#Step 3: Perform PCA down to 2D for visualization
pca = PCA(n_components = 4)
Y = pca.fit_transform(X)
eigs = pca.explained_variance_
print("lambda1 = %g, lambda2 = %g"%(eigs[0], eigs[1]))

#Step 4: Plot original signal and the persistence diagram
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(121)
ax.plot(t, x)
ax.set_ylim((-2, 2))
ax.set_title("Original Signal")
ax.set_xlabel("Phase")

ax2 = fig.add_subplot(122)
plotDGM(PDs[1])
plt.show()

37950 / 37950 possible edges added (100%)
